In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

import random
import tensorflow as tf
import keras 
from keras.models import Sequential
from keras.layers import Dense , Dropout , Conv1D , Flatten , LSTM
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings('ignore')


C:\Users\amgo02\Anaconda3\envs\old_laptop_unstable\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\amgo02\Anaconda3\envs\old_laptop_unstable\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\amgo02\Anaconda3\envs\old_laptop_unstable\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\amgo02\Anaconda3\en

In [2]:
train_mse_pointing = []
train_mean_relative_width_pointing = []
cross_val_mse_pointing = []
cross_val_mean_relative_width_pointing = []
val_mse_pointing = []
val_mean_relative_width_pointing = []
test_mse_pointing = []
test_mean_relative_width_pointing = []

train_mse_gaze = []
train_mean_relative_width_gaze = []
cross_val_mse_gaze = []
cross_val_mean_relative_width_gaze = []
val_mse_gaze = []
val_mean_relative_width_gaze = []
test_mse_gaze = []
test_mean_relative_width_gaze = []

train_mse_fuse = []
train_mean_relative_width_fuse = []
cross_val_mse_fuse = []
cross_val_mean_relative_width_fuse = []
val_mse_fuse = []
val_mean_relative_width_fuse = []
test_mse_fuse = []
test_mean_relative_width_fuse = []

train_std_relative_width_pointing = []
cross_val_std_relative_width_pointing = []
val_std_relative_width_pointing = []
test_std_relative_width_pointing = []

train_std_relative_width_gaze = []
cross_val_std_relative_width_gaze = []
val_std_relative_width_gaze = []
test_std_relative_width_gaze = []

train_std_relative_width_fuse = []
cross_val_std_relative_width_fuse = []
val_std_relative_width_fuse = []
test_std_relative_width_fuse = []

train_support_vector_shape_pointing = []
cross_val_support_vector_shape_pointing = []
val_support_vector_shape_pointing = []
test_support_vector_shape_pointing = []

train_support_vector_shape_gaze = []
cross_val_support_vector_shape_gaze = []
val_support_vector_shape_gaze = []
test_support_vector_shape_gaze = []

train_support_vector_shape_fuse = []
cross_val_support_vector_shape_fuse = []
val_support_vector_shape_fuse = []
test_support_vector_shape_fuse = []


FCNN_test_mse_pointing = []
FCNN_test_mean_relative_width_pointing = []
FCNN_test_mse_gaze = []
FCNN_test_mean_relative_width_gaze = []
FCNN_test_mse_fuse = []
FCNN_test_mean_relative_width_fuse = []
FCNN_test_std_relative_width_pointing = []
FCNN_test_std_relative_width_gaze = []
FCNN_test_std_relative_width_fuse = []

In [3]:
sync_confirmed_list_train_all = [[23,27,28,29,30,32,33,37,38,39,40,41,42,45,46,50,53,56,57,59,62,63,66,67,68,69,71,72,73],
                                [9,10,11,14,15,17,18,24,25,29,31,39,40,41,42,50,53,56,57,59,62,63,66,67,68,69,71,72,73],
                                [9,10,11,14,15,17,18,23,24,25,27,28,30,31,32,33,37,38,45,46,56,57,62,67,68,69,71,72,73],
                                [9,10,11,14,15,17,18,23,24,25,27,28,29,30,31,32,33,37,38,39,40,41,42,45,46,50,53,59,63,66],
                                [11,17,18,24,30,39,40,63,66,67,68,69,72],
                                [32,33,38,24,30,39,40,63,66,67,68,69,72],
                                [11,17,18,32,33,38,40,63,66,67,68,69,72],
                                [11,17,18,32,33,38,24,30,39,67,68,69,72],
                                [11,17,18,32,33,38,24,30,39,40,63,66,72],
                                [9,10,14,23,41,42,56,57,73],
                                [27,28,29,23,41,42,56,57,73],
                                [27,28,29,9,10,14,56,57,73],
                                [27,28,29,9,10,14,23,41,42],
                                [15,25,31,50,53,59,62,71],
                                [37,45,46,50,53,59,62,71],
                                [37,45,46,15,25,31,62,71],
                                [37,45,46,15,25,31,50,53]]
sync_confirmed_list_test_all = [[9,10,11,14,15,17,18,24,25,31],
                               [23,27,28,30,32,33,37,38,45,46],
                               [29,39,40,41,42,50,53,59,63,66],
                               [56,57,62,67,68,69,71,72,73],
                               [32,33,38],
                               [11,17,18],
                               [24,30,39],
                               [40,63,66],
                               [67,68,69],
                               [27,28,29],
                               [9,10,14],
                               [23,41,42],
                               [56,57,73],
                               [37,45,46],
                               [15,25,31],
                               [50,53,59],
                               [59,62,71]]

In [4]:
sync_confirmed_list_train = sync_confirmed_list_train_all[0]
sync_confirmed_list_test = sync_confirmed_list_test_all[0]

Participant_file_name = 'ML-PersRef_ML_Final_dataset/datapoints_rawdata_middle_point_with_sequence_with_building_width_'

dataset = []
dataset_test = []

for z in sync_confirmed_list_train:
    dataset.append(pd.read_csv(Participant_file_name + str(z)+'.csv'))

for j in sync_confirmed_list_test:
    dataset_test.append(pd.read_csv(Participant_file_name + str(j)+'.csv'))

dataset = pd.concat(dataset).reset_index(drop=True)
dataset_test = pd.concat(dataset_test).reset_index(drop=True)

In [5]:
def data_generator(sync_confirmed_list_train,sync_confirmed_list_test,
                   scaler,Participant_file_name,method,time_seq):

    dataset = []
    dataset_test = []

    for z in sync_confirmed_list_train:
        dataset.append(pd.read_csv(Participant_file_name + str(z)+'.csv'))

    for j in sync_confirmed_list_test:
        dataset_test.append(pd.read_csv(Participant_file_name + str(j)+'.csv'))

    dataset = pd.concat(dataset).reset_index(drop=True)
    dataset_test = pd.concat(dataset_test).reset_index(drop=True)

    if time_seq == "sequenced_model":
        
        triggers_to_remove = (((dataset.groupby(['Participant','trigger_no']).size().unstack(fill_value=0)) % sequence_no)!=0)
        triggers_to_remove = (triggers_to_remove.reset_index().melt(id_vars=['Participant']).query('value == True')).reset_index(drop=True)
        # print(dataset.values.astype(float).shape[0])

        dataset['column_to_check'] = dataset.Participant * 1000 + dataset.trigger_no
        triggers_to_remove['column_to_check'] = triggers_to_remove.Participant * 1000 + triggers_to_remove.trigger_no

        dataset = dataset[~(dataset.column_to_check.isin(triggers_to_remove.column_to_check))]

        groups = [dataset for _, dataset in dataset.groupby('column_to_check')]
        random.seed(42)
        random.shuffle(groups)

        dataset = pd.concat(groups).reset_index(drop=True)
        dataset.to_csv("dataset.csv")
        dataset.drop(columns=['column_to_check'],inplace=True)
        # print(dataset.values.astype(float).shape[0])

        triggers_to_remove = (((dataset_test.groupby(['Participant','trigger_no']).size().unstack(fill_value=0)) % sequence_no)!=0)
        triggers_to_remove = (triggers_to_remove.reset_index().melt(id_vars=['Participant']).query('value == True')).reset_index(drop=True)
        # print(dataset_test.values.astype(float).shape[0])

        dataset_test['column_to_check'] = dataset_test.Participant * 1000 + dataset_test.trigger_no
        triggers_to_remove['column_to_check'] = triggers_to_remove.Participant * 1000 + triggers_to_remove.trigger_no

        
        dataset_test = dataset_test[~(dataset_test.column_to_check.isin(triggers_to_remove.column_to_check))]
#         print(dataset_test.shape)
        
        groups = [dataset_test for _, dataset_test in dataset_test.groupby('column_to_check')]
        random.seed(42)
        random.shuffle(groups)
            
        dataset_test = pd.concat(groups).reset_index(drop=True)
        dataset_test.to_csv("dataset_test.csv")
        
        dataset_test.drop(columns=['column_to_check'],inplace=True)
#         print(dataset_test.shape)
        # print(dataset_test.values.astype(float).shape[0])

        print(((dataset.shape[0]//4)//sequence_no)*sequence_no)
        
        dataset_train = dataset.iloc[:-(((dataset.shape[0]//4)//sequence_no)*sequence_no),:]
        dataset_val = dataset.iloc[-(((dataset.shape[0]//4)//sequence_no)*sequence_no):,:]
        dataset_train.reset_index(inplace=True,drop=True)
        dataset_val.reset_index(inplace=True,drop=True)

    else:
        
        print("non_time_sequenced")
    
        dataset_train, dataset_val = train_test_split(dataset, test_size=0.25, random_state=42)
        dataset_train.reset_index(inplace=True,drop=True)
        dataset_val.reset_index(inplace=True,drop=True)
    
    if method == "No_gaze_removal_nor_GT_scaling":
        
        xtrain_cross_val = dataset[['fzd_hands0_fingerTip_coordinates_x',
               'fzd_hands0_fingerTip_coordinates_y',
               'fzd_hands0_fingerTip_coordinates_z', 'eyegaze_new_coordinates_x']].values.astype(float)
        ytrain_cross_val = dataset[['angle_center']].values.astype(float)
        ytrain_cross_val_width = dataset[['Buidling_half_width']].values.astype(float)
        
        xtrain = dataset_train[['fzd_hands0_fingerTip_coordinates_x',
               'fzd_hands0_fingerTip_coordinates_y',
               'fzd_hands0_fingerTip_coordinates_z', 'eyegaze_new_coordinates_x']].values.astype(float)
        ytrain = dataset_train[['angle_center']].values.astype(float)
        ytrain_width = dataset_train[['Buidling_half_width']].values.astype(float)

        xval = dataset_val[['fzd_hands0_fingerTip_coordinates_x',
               'fzd_hands0_fingerTip_coordinates_y',
               'fzd_hands0_fingerTip_coordinates_z', 'eyegaze_new_coordinates_x']].values.astype(float)
        yval = dataset_val[['angle_center']].values.astype(float)
        yval_width = dataset_val[['Buidling_half_width']].values.astype(float)

        xtest = dataset_test[['fzd_hands0_fingerTip_coordinates_x',
               'fzd_hands0_fingerTip_coordinates_y',
               'fzd_hands0_fingerTip_coordinates_z', 'eyegaze_new_coordinates_x']].values.astype(float)
        ytest = dataset_test[['angle_center']].values.astype(float)
        ytest_width = dataset_test[['Buidling_half_width']].values.astype(float)

        
    elif method == "gaze_removal_and_GT_scaling":
        
        dataset.dropna(inplace = True)
        dataset_train.dropna(inplace = True)
        dataset_val.dropna(inplace = True)
        dataset_test.dropna(inplace = True)
        
        xtrain_cross_val = dataset[['fzd_hands0_fingerTip_coordinates_x',
               'fzd_hands0_fingerTip_coordinates_y',
               'fzd_hands0_fingerTip_coordinates_z', 'eyegaze_new_coordinates_x_without_tablet_&_road_gazes']].values.astype(float)
        ytrain_cross_val = dataset[['angle_center_scaled']].values.astype(float)
        ytrain_cross_val_width = dataset[['Buidling_half_width_scaled']].values.astype(float)
        
        xtrain = dataset_train[['fzd_hands0_fingerTip_coordinates_x',
               'fzd_hands0_fingerTip_coordinates_y',
               'fzd_hands0_fingerTip_coordinates_z', 'eyegaze_new_coordinates_x_without_tablet_&_road_gazes']].values.astype(float)
        ytrain = dataset_train[['angle_center_scaled']].values.astype(float)
        ytrain_width = dataset_train[['Buidling_half_width_scaled']].values.astype(float)
        
        xtrain = dataset_train[['fzd_hands0_fingerTip_coordinates_x',
               'fzd_hands0_fingerTip_coordinates_y',
               'fzd_hands0_fingerTip_coordinates_z', 'eyegaze_new_coordinates_x_without_tablet_&_road_gazes']].values.astype(float)
        ytrain = dataset_train[['angle_center_scaled']].values.astype(float)
        ytrain_width = dataset_train[['Buidling_half_width_scaled']].values.astype(float)

        xval = dataset_val[['fzd_hands0_fingerTip_coordinates_x',
               'fzd_hands0_fingerTip_coordinates_y',
               'fzd_hands0_fingerTip_coordinates_z', 'eyegaze_new_coordinates_x_without_tablet_&_road_gazes']].values.astype(float)
        yval = dataset_val[['angle_center_scaled']].values.astype(float)
        yval_width = dataset_val[['Buidling_half_width_scaled']].values.astype(float)

        xtest = dataset_test[['fzd_hands0_fingerTip_coordinates_x',
               'fzd_hands0_fingerTip_coordinates_y',
               'fzd_hands0_fingerTip_coordinates_z', 'eyegaze_new_coordinates_x_without_tablet_&_road_gazes']].values.astype(float)
        ytest = dataset_test[['angle_center_scaled']].values.astype(float)
        ytest_width = dataset_test[['Buidling_half_width_scaled']].values.astype(float)
        

        
    elif method == "gaze_removal_but_no_GT_scaling":
        
        dataset.dropna(inplace = True)
        dataset_train.dropna(inplace = True)
        dataset_val.dropna(inplace = True)
        dataset_test.dropna(inplace = True)
        
        xtrain_cross_val = dataset[['fzd_hands0_fingerTip_coordinates_x',
               'fzd_hands0_fingerTip_coordinates_y',
               'fzd_hands0_fingerTip_coordinates_z', 'eyegaze_new_coordinates_x_without_tablet_&_road_gazes']].values.astype(float)
        ytrain_cross_val = dataset[['angle_center']].values.astype(float)
        ytrain_cross_val_width = dataset[['Buidling_half_width']].values.astype(float)
        
#         print(dataset)
        
        xtrain = dataset_train[['fzd_hands0_fingerTip_coordinates_x',
               'fzd_hands0_fingerTip_coordinates_y',
               'fzd_hands0_fingerTip_coordinates_z', 'eyegaze_new_coordinates_x_without_tablet_&_road_gazes']].values.astype(float)
        ytrain = dataset_train[['angle_center']].values.astype(float)
        ytrain_width = dataset_train[['Buidling_half_width']].values.astype(float)
        
        xval = dataset_val[['fzd_hands0_fingerTip_coordinates_x',
               'fzd_hands0_fingerTip_coordinates_y',
               'fzd_hands0_fingerTip_coordinates_z', 'eyegaze_new_coordinates_x_without_tablet_&_road_gazes']].values.astype(float)
        yval = dataset_val[['angle_center']].values.astype(float)
        yval_width = dataset_val[['Buidling_half_width']].values.astype(float)

        xtest = dataset_test[['fzd_hands0_fingerTip_coordinates_x',
               'fzd_hands0_fingerTip_coordinates_y',
               'fzd_hands0_fingerTip_coordinates_z', 'eyegaze_new_coordinates_x_without_tablet_&_road_gazes']].values.astype(float)
        ytest = dataset_test[['angle_center']].values.astype(float)
        ytest_width = dataset_test[['Buidling_half_width']].values.astype(float)

    sc_xtrain_cross_val = scaler
    xtrain_cross_val = sc_xtrain_cross_val.fit_transform(xtrain_cross_val)
    xtrain_cross_val_eye_gaze = xtrain_cross_val[:,-1].reshape(-1, 1)
    xtrain_cross_val_pointing = xtrain_cross_val[:,0:3]  
        
    sc_xtrain = scaler
    xtrain = sc_xtrain.fit_transform(xtrain)
    xtrain_eye_gaze = xtrain[:,-1].reshape(-1, 1)
    xtrain_pointing = xtrain[:,0:3]   
    
    sc_xval = scaler
    xval = sc_xtrain.transform(xval)
    xval_eye_gaze = xval[:,-1].reshape(-1, 1)
    xval_pointing = xval[:,0:3]
    
    sc_xtest = scaler
    xtest = sc_xtrain.transform(xtest)
    xtest_eye_gaze = xtest[:,-1].reshape(-1, 1)
    xtest_pointing = xtest[:,0:3]
    
    
#     print(xtrain_cross_val.shape, xtrain.shape, xval.shape, xtest.shape,
#       xtrain_cross_val_eye_gaze.shape, xtrain_eye_gaze.shape, xval_eye_gaze.shape, xtest_eye_gaze.shape,
#       xtrain_cross_val_pointing.shape, xtrain_pointing.shape, xval_pointing.shape, xtest_pointing.shape,
#       ytrain_cross_val.shape, ytrain.shape, yval.shape, ytest.shape)
    
    if time_seq == "sequenced_model":
        print(time_seq)
    
#         print(xtrain_cross_val[0:25,:])
#         print(ytrain_cross_val[0:25,:])
#         print("------")
        
        xtrain_cross_val = xtrain_cross_val.reshape(xtrain_cross_val.shape[0]//sequence_no,sequence_no,xtrain_cross_val.shape[1])
        xtrain_cross_val_eye_gaze = xtrain_cross_val_eye_gaze.reshape(xtrain_cross_val_eye_gaze.shape[0]//sequence_no,sequence_no,xtrain_cross_val_eye_gaze.shape[1])
        xtrain_cross_val_pointing = xtrain_cross_val_pointing.reshape(xtrain_cross_val_pointing.shape[0]//sequence_no,sequence_no,xtrain_cross_val_pointing.shape[1])

        xtrain = xtrain.reshape(xtrain.shape[0]//sequence_no,sequence_no,xtrain.shape[1])
        xtrain_eye_gaze = xtrain_eye_gaze.reshape(xtrain_eye_gaze.shape[0]//sequence_no,sequence_no,xtrain_eye_gaze.shape[1])
        xtrain_pointing = xtrain_pointing.reshape(xtrain_pointing.shape[0]//sequence_no,sequence_no,xtrain_pointing.shape[1])

        xval = xval.reshape(xval.shape[0]//sequence_no,sequence_no,xval.shape[1])
        xval_eye_gaze = xval_eye_gaze.reshape(xval_eye_gaze.shape[0]//sequence_no,sequence_no,xval_eye_gaze.shape[1])
        xval_pointing = xval_pointing.reshape(xval_pointing.shape[0]//sequence_no,sequence_no,xval_pointing.shape[1])

        xtest = xtest.reshape(xtest.shape[0]//sequence_no,sequence_no,xtest.shape[1])
        xtest_eye_gaze = xtest_eye_gaze.reshape(xtest_eye_gaze.shape[0]//sequence_no,sequence_no,xtest_eye_gaze.shape[1])
        xtest_pointing = xtest_pointing.reshape(xtest_pointing.shape[0]//sequence_no,sequence_no,xtest_pointing.shape[1])
        
#         print(xtrain_cross_val.shape, xtrain.shape, xval.shape, xtest.shape,
#               xtrain_cross_val_eye_gaze.shape, xtrain_eye_gaze.shape, xval_eye_gaze.shape, xtest_eye_gaze.shape,
#               xtrain_cross_val_pointing.shape, xtrain_pointing.shape, xval_pointing.shape, xtest_pointing.shape,
#               ytrain_cross_val.shape, ytrain.shape, yval.shape, ytest.shape)
        
        ytrain_cross_val = ytrain_cross_val.reshape(ytrain_cross_val.shape[0]//sequence_no,sequence_no,ytrain_cross_val.shape[1]) 
        ytrain = ytrain.reshape(ytrain.shape[0]//sequence_no,sequence_no,ytrain.shape[1])
        yval = yval.reshape(yval.shape[0]//sequence_no,sequence_no,yval.shape[1]) 
        ytest = ytest.reshape(ytest.shape[0]//sequence_no,sequence_no,ytest.shape[1]) 
        
        ytrain_cross_val_width = ytrain_cross_val_width.reshape(ytrain_cross_val_width.shape[0]//sequence_no,sequence_no,ytrain_cross_val_width.shape[1]) 
        ytrain_width = ytrain_width.reshape(ytrain_width.shape[0]//sequence_no,sequence_no,ytrain_width.shape[1])
        yval_width = yval_width.reshape(yval_width.shape[0]//sequence_no,sequence_no,yval_width.shape[1])
        ytest_width = ytest_width.reshape(ytest_width.shape[0]//sequence_no,sequence_no,ytest_width.shape[1])
        
#         print(xtrain_cross_val.shape, xtrain.shape, xval.shape, xtest.shape,
#               xtrain_cross_val_eye_gaze.shape, xtrain_eye_gaze.shape, xval_eye_gaze.shape, xtest_eye_gaze.shape,
#               xtrain_cross_val_pointing.shape, xtrain_pointing.shape, xval_pointing.shape, xtest_pointing.shape,
#               ytrain_cross_val.shape, ytrain.shape, yval.shape, ytest.shape)
        
#         print("------")
        
#         print(xtrain_cross_val[1,:,:])
#         print(ytrain_cross_val[1,:,:])
#         print(np.mean(ytrain_cross_val[1,:,:]))
#         print(np.median(ytrain_cross_val[1,:,:]))
#         print("median axis")
#         print(np.median(ytrain_cross_val,axis=1))
#         print((np.median(ytrain_cross_val,axis=1)).shape)
#         print("median axis")
        
        ytrain_cross_val = np.median(ytrain_cross_val,axis=1)
        ytrain = np.median(ytrain,axis=1)
        yval = np.median(yval,axis=1)
        ytest = np.median(ytest,axis=1)
        
        ytrain_cross_val_width = np.median(ytrain_cross_val_width,axis=1)
        ytrain_width = np.median(ytrain_width,axis=1)
        yval_width = np.median(yval_width,axis=1)
        ytest_width = np.median(ytest_width,axis=1)
        
#         print(xtrain_cross_val[1,:,:])
#         print(ytrain_cross_val[1,:,:])
#         print(np.mean(ytrain_cross_val[1,:,:]))
        


    print(xtrain_cross_val.shape, xtrain.shape, xval.shape, xtest.shape,
      xtrain_cross_val_eye_gaze.shape, xtrain_eye_gaze.shape, xval_eye_gaze.shape, xtest_eye_gaze.shape,
      xtrain_cross_val_pointing.shape, xtrain_pointing.shape, xval_pointing.shape, xtest_pointing.shape,
      ytrain_cross_val.shape, ytrain.shape, yval.shape, ytest.shape)
    
    
    
    return xtrain_cross_val, xtrain, xval, xtest, xtrain_cross_val_eye_gaze, \
            xtrain_eye_gaze, xval_eye_gaze, xtest_eye_gaze,xtrain_cross_val_pointing, \
            xtrain_pointing, xval_pointing, xtest_pointing,ytrain_cross_val, ytrain, yval, ytest, \
            ytrain_cross_val_width, ytrain_width, yval_width, ytest_width

In [6]:
def LSTM_model(sequence_no,dim_num):
    
    #create model
    model = Sequential()
    #add model layers
    model.add(LSTM(10, input_shape=(sequence_no,dim_num)))
    model.add(Dense(1,activation='linear'))
    print(model.summary())
    # Compile model
    # optimizerr = keras.optimizers.SGD(lr=0.001)
    model.compile(loss='mse', optimizer='adam',metrics=['mse'])
    return model


def CNN_model(sequence_no,dim_num):
    
    #create model
    model = Sequential()
    #add model layers
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(sequence_no,dim_num)))
#     model.add(Conv1D(8, kernel_size=1, activation='relu'))
#     model.add(Conv2D(32, kernel_size=3, activation=’relu’))
    model.add(Flatten())
    model.add(Dense(1,activation='linear'))
    print(model.summary())
    # Compile model
    # optimizerr = keras.optimizers.SGD(lr=0.001)
    model.compile(loss='mse', optimizer='adam',metrics=['mse'])
    return model

def baseline_model(dim_num):
    # create model
    model = Sequential()
    model.add(Dense(32, input_dim=dim_num,  activation='relu', kernel_initializer='uniform'))
#     model.add(Dropout(0.5))
    model.add(Dense(16, activation='relu', kernel_initializer='uniform'))
#     model.add(Dropout(0.5))
    model.add(Dense(8, activation='relu', kernel_initializer='uniform'))
#     model.add(Dropout(0.5))
#     model.add(Dense(4, activation='relu', kernel_initializer='normal'))
#     model.add(Dropout(0.5))
    model.add(Dense(1,activation='linear'))
#     print(model.summary())
    # Compile model
    # optimizerr = keras.optimizers.SGD(lr=0.001)
    model.compile(loss='mse', optimizer='adam',metrics=['mse'])
    return model

def Main_Network_caller(xtrain1, ytrain1,xval1, yval1,xtest1):
    checkpoint_name = 'Weights-best.hdf5' 
    checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 0, save_best_only = True, mode ='auto')
    callbacks_list = [checkpoint]

    dim_num = xtrain1.shape[-1]
    
    if type_ == "FCNN":
        NN_model = baseline_model(dim_num)
    elif type_ == "CNN":
        NN_model = CNN_model(sequence_no,dim_num)
    elif type_ == "LSTM":
        NN_model = LSTM_model(sequence_no,dim_num)
    
    NN_model.fit(xtrain1, ytrain1, epochs=1000, batch_size=32,
                 validation_data = (xval1, yval1), callbacks=callbacks_list, verbose = 0)

    # Load wights file of the best model :
    wights_file = 'Weights-best.hdf5' # choose the best checkpoint 
    NN_model.load_weights(wights_file) # load it
    NN_model.compile(loss='mse', optimizer='adam', metrics=['mse'])

    pred = NN_model.predict(xtest1)
    return pred

In [7]:
def results_calculator(true_value,prediction,relative_width,tolerance_degree):
    
    mse = mean_squared_error(true_value, prediction)
    relative_width = abs(true_value-prediction)/(relative_width + tolerance_degree)
    return mse, relative_width
    

In [8]:
def regressor(type_,eps):
    
    xtrain_cross_val, xtrain, xval, xtest, xtrain_cross_val_eye_gaze, \
    xtrain_eye_gaze, xval_eye_gaze, xtest_eye_gaze,xtrain_cross_val_pointing, \
    xtrain_pointing, xval_pointing, xtest_pointing,ytrain_cross_val, \
    ytrain, yval, ytest, ytrain_cross_val_width, ytrain_width, \
    yval_width, ytest_width = data_generator(sync_confirmed_list_train,
                                         sync_confirmed_list_test,scaler,Participant_file_name,method,time_seq)
    
    #print('----------------------')
    #print("----train_cross_val----")
    #print('----------------------')
    
    if type_ == "SVR":
        
        print(type_)
#         print(eps)
        regressor = SVR(kernel='rbf', epsilon=eps)

        regressor.fit(xtrain_cross_val,ytrain_cross_val)
        cross_val_support_vector_shape_fuse.append(regressor.support_vectors_.shape)
        pred = cross_val_predict(regressor, xtrain_cross_val, ytrain_cross_val).reshape(-1, 1)
        mse, relative_width = results_calculator(ytrain_cross_val,pred,ytrain_cross_val_width,tolerance_degree)

        cross_val_mse_fuse.append(mse)
        cross_val_mean_relative_width_fuse.append(relative_width.mean())
        cross_val_std_relative_width_fuse.append(relative_width.std())

        regressor.fit(xtrain_cross_val_pointing,ytrain_cross_val)
        cross_val_support_vector_shape_pointing.append(regressor.support_vectors_.shape)
        pred = cross_val_predict(regressor, xtrain_cross_val_pointing, ytrain_cross_val).reshape(-1, 1)
        mse, relative_width = results_calculator(ytrain_cross_val,pred,ytrain_cross_val_width,tolerance_degree)

        cross_val_mse_pointing.append(mse)
        cross_val_mean_relative_width_pointing.append(relative_width.mean())
        cross_val_std_relative_width_pointing.append(relative_width.std())

        regressor.fit(xtrain_cross_val_eye_gaze,ytrain_cross_val)
        cross_val_support_vector_shape_gaze.append(regressor.support_vectors_.shape)
        pred = cross_val_predict(regressor, xtrain_cross_val_eye_gaze, ytrain_cross_val).reshape(-1, 1)
        mse, relative_width = results_calculator(ytrain_cross_val,pred,ytrain_cross_val_width,tolerance_degree)

        cross_val_mse_gaze.append(mse)
        cross_val_mean_relative_width_gaze.append(relative_width.mean())
        cross_val_std_relative_width_gaze.append(relative_width.std())

        #print('=========================')
        #print('------------------------')
        #print("---------train----------")
        #print('------------------------')

        regressor.fit(xtrain,ytrain)
        train_support_vector_shape_fuse.append(regressor.support_vectors_.shape)
        pred=regressor.predict(xtrain).reshape(-1, 1)
        mse, relative_width = results_calculator(ytrain,pred,ytrain_width,tolerance_degree)
        xtrain_fuse = regressor.support_vectors_
        ytrain_fuse = ytrain[regressor.support_]

        train_mse_fuse.append(mse)
        train_mean_relative_width_fuse.append(relative_width.mean())
        train_std_relative_width_fuse.append(relative_width.std())

        regressor.fit(xtrain_pointing,ytrain)
        train_support_vector_shape_pointing.append(regressor.support_vectors_.shape)
        pred=regressor.predict(xtrain_pointing).reshape(-1, 1)
        mse, relative_width = results_calculator(ytrain,pred,ytrain_width,tolerance_degree)

        xtrain_pointing = regressor.support_vectors_
        ytrain_pointing = ytrain[regressor.support_]

        train_mse_pointing.append(mse)
        train_mean_relative_width_pointing.append(relative_width.mean())
        train_std_relative_width_pointing.append(relative_width.std())

        regressor.fit(xtrain_eye_gaze,ytrain)
        train_support_vector_shape_gaze.append(regressor.support_vectors_.shape)
        pred=regressor.predict(xtrain_eye_gaze).reshape(-1, 1)
        mse, relative_width = results_calculator(ytrain,pred,ytrain_width,tolerance_degree)

        xtrain_eye_gaze = regressor.support_vectors_
        ytrain_eye_gaze = ytrain[regressor.support_]

        train_mse_gaze.append(mse)
        train_mean_relative_width_gaze.append(relative_width.mean())
        train_std_relative_width_gaze.append(relative_width.std())

        #print('----------------------')
        #print("---------val----------")
        #print('----------------------')

        regressor.fit(xval,yval)
        val_support_vector_shape_fuse.append(regressor.support_vectors_.shape)
        pred=regressor.predict(xval).reshape(-1, 1)
        mse, relative_width = results_calculator(yval,pred,yval_width,tolerance_degree)

        val_mse_fuse.append(mse)
        val_mean_relative_width_fuse.append(relative_width.mean())
        val_std_relative_width_fuse.append(relative_width.std())

        regressor.fit(xval_pointing,yval)
        val_support_vector_shape_pointing.append(regressor.support_vectors_.shape)
        pred=regressor.predict(xval_pointing).reshape(-1, 1)
        mse, relative_width = results_calculator(yval,pred,yval_width,tolerance_degree)

        val_mse_pointing.append(mse)
        val_mean_relative_width_pointing.append(relative_width.mean())
        val_std_relative_width_pointing.append(relative_width.std())

        regressor.fit(xval_eye_gaze,yval)
        val_support_vector_shape_gaze.append(regressor.support_vectors_.shape)
        pred=regressor.predict(xval_eye_gaze).reshape(-1, 1)
        mse, relative_width = results_calculator(yval,pred,yval_width,tolerance_degree)

        val_mse_gaze.append(mse)
        val_mean_relative_width_gaze.append(relative_width.mean())
        val_std_relative_width_gaze.append(relative_width.std())

        #print('------------------------')
        #print("----------test----------")
        #print('------------------------')

        regressor.fit(xtrain_fuse,ytrain_fuse)
        test_support_vector_shape_fuse.append(regressor.support_vectors_.shape)
        pred=regressor.predict(xtest).reshape(-1, 1)
        mse, relative_width = results_calculator(ytest,pred,ytest_width,tolerance_degree)

        test_mse_fuse.append(mse)
        test_mean_relative_width_fuse.append(relative_width.mean())
        test_std_relative_width_fuse.append(relative_width.std())

        regressor.fit(xtrain_pointing,ytrain_pointing)
        test_support_vector_shape_pointing.append(regressor.support_vectors_.shape)
        pred=regressor.predict(xtest_pointing).reshape(-1, 1)
        mse, relative_width = results_calculator(ytest,pred,ytest_width,tolerance_degree)

        test_mse_pointing.append(mse)
        test_mean_relative_width_pointing.append(relative_width.mean())
        test_std_relative_width_pointing.append(relative_width.std())

        regressor.fit(xtrain_eye_gaze,ytrain_eye_gaze)
        test_support_vector_shape_gaze.append(regressor.support_vectors_.shape)
        pred=regressor.predict(xtest_eye_gaze).reshape(-1, 1)
        mse, relative_width = results_calculator(ytest,pred,ytest_width,tolerance_degree)

        test_mse_gaze.append(mse)
        test_mean_relative_width_gaze.append(relative_width.mean())
        test_std_relative_width_gaze.append(relative_width.std())
        
#     elif type_ == "FCNN":
    else:
        
        print(type_)
        
        pred = Main_Network_caller(xtrain, ytrain,xval, yval,xtest)
        mse, relative_width = results_calculator(ytest,pred,ytest_width,tolerance_degree)
#         print('===================')
#         print('Test results fuse')
#         print('-------------------')
#         print(mse)
#         print(relative_width.mean())
#         print(relative_width.std())
        
        FCNN_test_mse_fuse.append(mse)
        FCNN_test_mean_relative_width_fuse.append(relative_width.mean())
        FCNN_test_std_relative_width_fuse.append(relative_width.std())
        
        pred = Main_Network_caller(xtrain_pointing, ytrain,xval_pointing, yval,xtest_pointing)
        mse, relative_width = results_calculator(ytest,pred,ytest_width,tolerance_degree)
#         print('===================')
#         print('Test results pointing')
#         print('-------------------')
#         print(mse)
#         print(relative_width.mean())
#         print(relative_width.std())
        
        FCNN_test_mse_pointing.append(mse)
        FCNN_test_mean_relative_width_pointing.append(relative_width.mean())
        FCNN_test_std_relative_width_pointing.append(relative_width.std())
        
        pred = Main_Network_caller(xtrain_eye_gaze, ytrain,xval_eye_gaze, yval,xtest_eye_gaze)
        mse, relative_width = results_calculator(ytest,pred,ytest_width,tolerance_degree)
#         print('===================')
#         print('Test results gaze')
#         print('-------------------')
#         print(mse)
#         print(relative_width.mean())
#         print(relative_width.std())
        
        FCNN_test_mse_gaze.append(mse)
        FCNN_test_mean_relative_width_gaze.append(relative_width.mean())
        FCNN_test_std_relative_width_gaze.append(relative_width.std())
        

In [13]:
# for i in range(1):
for i in range(len(sync_confirmed_list_train_all)):
    print(i)
    sequence_no = 7
    
    sync_confirmed_list_train = sync_confirmed_list_train_all[i]
    sync_confirmed_list_test = sync_confirmed_list_test_all[i]
#     print(sync_confirmed_list_train)
#     print(sync_confirmed_list_test)
    
    # eps = 12
    scaler = StandardScaler() #MinMaxScaler()
    tolerance_degree = 0

#     Participant_file_name = 'ML-PersRef_ML_Final_dataset/datapoints_rawdata_first_middle_points_with_sequence_' + str(sequence_no) + '_with_building_width_'
    Participant_file_name = 'ML-PersRef_ML_Final_dataset/datapoints_rawdata_first_middle_points_with_building_width_'
    method = "No_gaze_removal_nor_GT_scaling"
#     method = "gaze_removal_but_no_GT_scaling"

    
#     time_seq = "sequenced_model"
    time_seq = 0


    type_ = "SVR"
    regressor(type_,2)

#     type_ = "FCNN"
#     regressor(type_,0)
    
#     type_ = "CNN"
#     regressor(type_,0)
    
#     type_ = "LSTM"
#     regressor(type_,0)

    # for eps in np.arange(0, 7, 1):
    #     try:
    #         regressor("SVR",sync_confirmed_list_train,sync_confirmed_list_test,eps,scaler,tolerance_degree,Participant_file_name,method)
    #     except:
    #         break

0
non_time_sequenced
(3621, 4) (2715, 4) (906, 4) (1309, 4) (3621, 1) (2715, 1) (906, 1) (1309, 1) (3621, 3) (2715, 3) (906, 3) (1309, 3) (3621, 1) (2715, 1) (906, 1) (1309, 1)
SVR
1
non_time_sequenced
(3750, 4) (2812, 4) (938, 4) (1180, 4) (3750, 1) (2812, 1) (938, 1) (1180, 1) (3750, 3) (2812, 3) (938, 3) (1180, 3) (3750, 1) (2812, 1) (938, 1) (1180, 1)
SVR
2
non_time_sequenced
(3657, 4) (2742, 4) (915, 4) (1273, 4) (3657, 1) (2742, 1) (915, 1) (1273, 1) (3657, 3) (2742, 3) (915, 3) (1273, 3) (3657, 1) (2742, 1) (915, 1) (1273, 1)
SVR
3
non_time_sequenced
(3762, 4) (2821, 4) (941, 4) (1168, 4) (3762, 1) (2821, 1) (941, 1) (1168, 1) (3762, 3) (2821, 3) (941, 3) (1168, 3) (3762, 1) (2821, 1) (941, 1) (1168, 1)
SVR
4
non_time_sequenced
(1761, 4) (1320, 4) (441, 4) (406, 4) (1761, 1) (1320, 1) (441, 1) (406, 1) (1761, 3) (1320, 3) (441, 3) (406, 3) (1761, 1) (1320, 1) (441, 1) (406, 1)
SVR
5
non_time_sequenced
(1734, 4) (1300, 4) (434, 4) (433, 4) (1734, 1) (1300, 1) (434, 1) (433, 1) (1

In [14]:

print(test_support_vector_shape_fuse)
print(test_mse_fuse)
print(test_mean_relative_width_fuse)
print(test_std_relative_width_fuse)
print(test_support_vector_shape_pointing)
print(test_mse_pointing)
print(test_mean_relative_width_pointing)
print(test_std_relative_width_pointing)
print(test_support_vector_shape_gaze)
print(test_mse_gaze)
print(test_mean_relative_width_gaze)
print(test_std_relative_width_gaze)


[(1544, 4), (1723, 4), (1509, 4), (1668, 4), (820, 4), (724, 4), (742, 4), (747, 4), (728, 4), (458, 4), (394, 4), (437, 4), (409, 4), (487, 4), (494, 4), (403, 4), (497, 4)]
[149.2975715688998, 222.2981280097404, 374.4521679164791, 233.50806359496795, 116.33246941697958, 113.26594337301208, 200.9010920700587, 236.26773567862023, 160.4509505231597, 121.56642812090458, 142.18141365120408, 131.30769080351095, 241.8078583314413, 534.1282903960662, 200.7230018616501, 880.8534967164256, 582.3285384178746]
[1.868636728133873, 1.8106377049338218, 2.2085082884202216, 1.7178944841791666, 1.6459859015273237, 1.8328613574254127, 2.1187631354361254, 2.3342068281661272, 1.6761928900993965, 1.4602428688255589, 2.16136664359862, 1.59999362820438, 1.9499242856810883, 3.3655993246407, 2.330441837393549, 3.145968088465969, 2.0680806960418274]
[2.3764289773819782, 3.421331962176346, 3.2955015367846943, 3.395046572761664, 2.14237641641377, 2.486865705035996, 3.371114318511845, 3.12286961008506, 2.88411441